# Dataproc with BigLake Metastore

A guide to setting up a single-node Dataproc cluster with a BigLake Metastore.

**Note:** Steps 1-3 are for provisioning the cluster from your local environment.

## Step 1: Configure Cluster Settings

Set the configuration for your Dataproc cluster. **Replace placeholders** with your Google Cloud project details.

In [ ]:
# CONFIG: Replace these values with your project details.
PROJECT_ID = "my-project-id"  # Your Google Cloud project ID
REGION = "us-central1"      # The region for the cluster
CLUSTER_NAME = "my-single-node-cluster"  # A name for your Dataproc cluster
BUCKET_NAME = f"{PROJECT_ID}-dataproc-bucket"  # A unique GCS bucket name

# Hive settings
HIVE_DB = "my_hive_db"
HIVE_TABLE = f"{HIVE_DB}.people_hive"
HIVE_GCS_TABLE = f"{HIVE_DB}.people_hive_gcs"

# Iceberg on Hive settings
ICEBERG_HIVE_CATALOG = "iceberg_on_hive"
ICEBERG_HIVE_DB = "my_iceberg_db"
ICEBERG_HIVE_TABLE = f"{ICEBERG_HIVE_CATALOG}.{ICEBERG_HIVE_DB}.people_iceberg"
ICEBERG_HIVE_FROM_BQ = f"{ICEBERG_HIVE_CATALOG}.{ICEBERG_HIVE_DB}.people_filtered_bq"
ICEBERG_HIVE_FROM_SPARK = f"{ICEBERG_HIVE_CATALOG}.{ICEBERG_HIVE_DB}.people_filtered_spark"

# Iceberg on BigLake Metastore settings
BIGLAKE_DATASET = "my_iceberg_metastore"
BIGLAKE_TABLE = f"{BIGLAKE_DATASET}.people_biglake"
ICEBERG_BIGLAKE_CATALOG = "iceberg_on_bq"
ICEBERG_BIGLAKE_TABLE = f"{ICEBERG_BIGLAKE_CATALOG}.{BIGLAKE_TABLE}"
ICEBERG_BIGLAKE_FROM_SPARK = f"{ICEBERG_BIGLAKE_CATALOG}.{BIGLAKE_DATASET}.people_filtered_spark"

# # Trino on GCE settings
# GCE_INSTANCE_TRINO = "trino-biglake-vm"
# GCE_ZONE_TRINO = "us-central1-a"
GCE_INSTANCE_SPARK = "spark-biglake-vm"
GCE_ZONE_SPARK = "us-central1-a"

## Step 2: Create and Upload Initialization Script

This script downloads Iceberg and BigQuery JARs and places them in Spark's classpath. It's uploaded to a GCS bucket for use during cluster creation.

In [ ]:
# This script downloads required JARs for Iceberg on BigLake Metastore integration.
init_script_content = """#!/bin/bash
# install-jars.sh
set -e -x

# URLs for the JAR files
ICEBERG_RUNTIME_URL="https://storage-download.googleapis.com/maven-central/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.6.1/iceberg-spark-runtime-3.5_2.12-1.6.1.jar"
BQ_CATALOG_JAR_GCS="gs://spark-lib/bigquery/iceberg-bigquery-catalog-1.6.1-1.0.1-beta.jar"

# Download JARs to Spark's classpath
wget -P /usr/lib/spark/jars/ "$ICEBERG_RUNTIME_URL"
gsutil cp "$BQ_CATALOG_JAR_GCS" /usr/lib/spark/jars/
"""

# Define local and GCS paths
local_script_path = "install-jars.sh"
gcs_script_path = f"gs://{BUCKET_NAME}/scripts/{local_script_path}"

# Write the script to a local file
with open(local_script_path, "w") as f:
    f.write(init_script_content)

print(f"Initialization script created at: {local_script_path}")

# Upload the script to GCS
!gsutil cp {local_script_path} {gcs_script_path}

print(f"Successfully uploaded script to: {gcs_script_path}")

## Step 3: Create Dataproc Cluster with Iceberg Catalog

This command creates a Dataproc cluster with the necessary Spark properties for Iceberg. Setting these at the cluster level ensures the Spark session is pre-configured.

In [ ]:
# Define properties for the Dataproc cluster
properties_list = [
    "spark:spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    f"spark:spark.sql.catalog.{ICEBERG_HIVE_CATALOG}=org.apache.iceberg.spark.SparkCatalog",
    f"spark:spark.sql.catalog.{ICEBERG_HIVE_CATALOG}.type=hive",
    f"spark:spark.sql.catalog.{ICEBERG_HIVE_CATALOG}.warehouse=hdfs:///user/iceberg_on_hive",
    f"spark:spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}=org.apache.iceberg.spark.SparkCatalog",
    f"spark:spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.catalog-impl=org.apache.iceberg.gcp.bigquery.BigQueryMetastoreCatalog",
    f"spark:spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.gcp_project={PROJECT_ID}",
    f"spark:spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.location={REGION}",
    f"spark:spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.warehouse=gs://{BUCKET_NAME}/{ICEBERG_BIGLAKE_CATALOG}"
]
PROPERTIES = ",".join(properties_list)

# Create the Dataproc cluster
!gcloud dataproc clusters create {CLUSTER_NAME} \
    --project {PROJECT_ID} \
    --region {REGION} \
    --single-node \
    --image-version 2.2-debian12 \
    --optional-components=JUPYTER \
    --enable-component-gateway \
    --bucket {BUCKET_NAME} \
    --initialization-actions={gcs_script_path} \
    --properties="{PROPERTIES}"

print(f"Cluster '{CLUSTER_NAME}' creation initiated.")

## Step 4: Access Jupyter and Get Spark Session

Once the cluster is running, access the Jupyter environment. The Spark session is pre-configured, so no extra setup is needed.

In [ ]:
# Get the pre-configured Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

print("Spark session is active and ready to use.")

## Step 5: Create a Sample DataFrame and Write to HDFS

Create a sample DataFrame and write it to HDFS as a Parquet file.

In [ ]:
# Create a sample DataFrame
df = spark.createDataFrame(
    [('Alice', 25), ('Bob', 30), ('Charlie', 35)], ['name', 'age'])

# Write the DataFrame to HDFS
df.write.mode('overwrite').parquet('/user/my_data/people')

# Verify the file was created in HDFS
!hdfs dfs -ls /user/my_data/people

---

## Scenario 1: Create a Hive Table from HDFS

Create a Hive table from the data in HDFS.

In [ ]:
# Create a Hive database
spark.sql(
    f"CREATE DATABASE IF NOT EXISTS {HIVE_DB} LOCATION 'hdfs:///user/hive_db'")

# Drop the table if it already exists
spark.sql(f"DROP TABLE IF EXISTS {HIVE_TABLE}")

# Create an external Hive table pointing to the HDFS data
spark.sql(f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {HIVE_TABLE} (
        name STRING,
        age BIGINT
    )
    STORED AS PARQUET
    LOCATION '/user/my_data/people'
""")

print("--- Standard Hive Table ---")
# Query the Hive table
spark.sql(f"SELECT * FROM {HIVE_TABLE}").show()

## Scenario 2: Create an Iceberg Table from HDFS

Create an Iceberg table using the Hive metastore.

In [ ]:
print("Verifying the Iceberg catalog...")
# Show databases in the Hive Iceberg catalog
spark.sql(f"SHOW DATABASES IN {ICEBERG_HIVE_CATALOG}").show()

# Create a database in the Hive Iceberg catalog
spark.sql(
    f"CREATE DATABASE IF NOT EXISTS {ICEBERG_HIVE_CATALOG}.{ICEBERG_HIVE_DB}")

# Write the DataFrame to an Iceberg table
spark.sql(f"SELECT * FROM {HIVE_TABLE}") \
    .write \
    .format("iceberg") \
    .mode("overwrite") \
    .saveAsTable(ICEBERG_HIVE_TABLE)

print("--- Iceberg Table on Hive Metastore ---")
# Query the Iceberg table
spark.sql(f"SELECT * FROM {ICEBERG_HIVE_TABLE}").show()

---

## Scenario 3: Create a Hive (or Iceberg) Table from GCS

Copy the Parquet file to GCS and create a Hive table from it.

In [ ]:
# Define GCS path and create the GCS directory if needed
GCS_PATH = f"gs://{BUCKET_NAME}/hive_on_gcs/data"
!touch _CF && gsutil cp -r _CF {GCS_PATH}/ && rm _CF

# Copy the HDFS data to GCS
!hdfs dfs -cp /user/my_data/people {GCS_PATH}/{HIVE_GCS_TABLE}

# Drop the table if it already exists
spark.sql(f"DROP TABLE IF EXISTS {HIVE_GCS_TABLE}")

# Create an external Hive table pointing to the GCS data
spark.sql(f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {HIVE_GCS_TABLE} (
        name STRING,
        age BIGINT
    )
    STORED AS PARQUET
    LOCATION '{GCS_PATH}/{HIVE_GCS_TABLE}'
""")

print("--- Hive Table on GCS ---")
# Query the Hive table
spark.sql(f"SELECT * FROM {HIVE_GCS_TABLE}").show()

## Scenario 4: Create an Iceberg Table with BigLake Metastore

Create an Iceberg table using the BigLake metastore.

In [ ]:
print("Verifying the BigQuery catalog...")
# Show databases in the BigQuery Iceberg catalog
spark.sql(f"SHOW DATABASES IN {ICEBERG_BIGLAKE_CATALOG}").show()

# Create the BigQuery dataset to act as the metastore
!bq mk --connection --location={REGION} --project_id={PROJECT_ID} --connection_type=CLOUD_RESOURCE default-{REGION}
!bq --location={REGION} mk --dataset {PROJECT_ID}:{BIGLAKE_DATASET}

# Drop the table if it already exists
spark.sql(f"DROP TABLE IF EXISTS {ICEBERG_BIGLAKE_TABLE}")

# Create the Iceberg table in the BigLake metastore
spark.sql(f"""
CREATE TABLE IF NOT EXISTS
  {ICEBERG_BIGLAKE_TABLE} ( name string,
    age int )
USING
  ICEBERG TBLPROPERTIES ('bq_connection'='projects/{PROJECT_ID}/locations/{REGION}/connections/default-{REGION}');
""")

# Save the DataFrame to the BigLake metastore
spark.sql(f"SELECT * FROM {ICEBERG_HIVE_TABLE}") \
    .write \
    .format("iceberg") \
    .mode("overwrite") \
    .save(ICEBERG_BIGLAKE_TABLE)

# Query the Iceberg table from Spark
# spark.sql(f"SELECT * FROM {ICEBERG_BIGLAKE_TABLE}").show()

sql_query = f"SELECT * FROM {BIGLAKE_TABLE}"
df = spark.read \
    .format("bigquery") \
    .option("viewsEnabled", "true") \
    .option("query", sql_query) \
    .option("materializationDataset", BIGLAKE_DATASET) \
    .load()
df.show()

---

## Scenario 5: Push Down Computation to BigQuery

Use the BigQuery Connector to execute a SQL query directly in BigQuery. Only the results are returned to Spark.

In [ ]:
# This query is executed directly in BigQuery
bq_sql_query = f"""
SELECT
    name,
    age
FROM
    {BIGLAKE_TABLE}
WHERE
    age > 28
"""

# Use the 'bigquery' format to send the query to BigQuery
filtered_df = spark.read \
    .format("bigquery") \
    .option("viewsEnabled", "true") \
    .option("query", bq_sql_query) \
    .option("materializationDataset", BIGLAKE_DATASET) \
    .load()

print(f"--- Data returned from BigQuery ---")
filtered_df.show()

# Save the filtered results to the Hive metastore
filtered_df.write.format("iceberg").mode(
    "overwrite").saveAsTable(ICEBERG_HIVE_FROM_BQ)

print("--- Iceberg Table on Hive Metastore ---")
spark.sql(f"SELECT * FROM {ICEBERG_HIVE_FROM_BQ}").show()

## Scenario 6: Push Down Computation with Serverless Spark

Use Serverless Spark to execute a query. The results are written to the Hive metastore.

In [ ]:
# Python script for the Spark job
pyspark_job_content = f"""
from pyspark.sql import SparkSession

# These values are injected by the gcloud command
ICEBERG_BIGLAKE_TABLE = "{ICEBERG_BIGLAKE_TABLE}"
ICEBERG_BIGLAKE_FROM_SPARK = f"{ICEBERG_BIGLAKE_FROM_SPARK}"

def main():
    spark = SparkSession.builder \\
        .appName("Dataproc Serverless Spark Filter") \\
        .getOrCreate()

    # This is a Spark SQL query, not a BigQuery query
    filter_query = f'''
    SELECT
        name,
        age
    FROM
        {{ICEBERG_BIGLAKE_TABLE}}
    WHERE
        age > 28
    '''

    print(f"--- Running Spark SQL query: {{filter_query}} ---")

    filtered_df = spark.sql(filter_query)

    print("--- Filtered data computed by Spark ---")
    filtered_df.show()

    # Save results to a new Iceberg table
    print(f"--- Saving results to: {{ICEBERG_BIGLAKE_FROM_SPARK}} ---")
    filtered_df.write \\
        .format("iceberg") \\
        .mode("overwrite") \\
        .saveAsTable(ICEBERG_BIGLAKE_FROM_SPARK)

    print("Job completed.")

if __name__ == "__main__":
    main()
"""

# Define local and GCS paths for the job script
local_job_path = "filter_job_spark_sql.py"
gcs_job_path = f"gs://{BUCKET_NAME}/scripts/{local_job_path}"

# Write and upload the script
with open(local_job_path, "w") as f:
    f.write(pyspark_job_content)

!gsutil cp {local_job_path} {gcs_job_path}

print(f"Successfully uploaded Spark job script to: {gcs_job_path}")

In [ ]:
# Define properties for the serverless Dataproc job
properties_list = [
    "spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    f"spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}=org.apache.iceberg.spark.SparkCatalog",
    f"spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.catalog-impl=org.apache.iceberg.gcp.bigquery.BigQueryMetastoreCatalog",
    f"spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.gcp_project={PROJECT_ID}",
    f"spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.location={REGION}",
    f"spark.sql.catalog.{ICEBERG_BIGLAKE_CATALOG}.warehouse=gs://{BUCKET_NAME}/{ICEBERG_BIGLAKE_CATALOG}"
]
PROPERTIES = ",".join(properties_list)

# Submit the PySpark job to a serverless Dataproc cluster
!gcloud dataproc batches submit pyspark {gcs_job_path} \
    --project={PROJECT_ID} \
    --region={REGION} \
    --batch="serverless-spark-engine-job" \
    --version="2.2" \
    --subnet="default" \
    --jars="gs://spark-lib/bigquery/iceberg-bigquery-catalog-1.6.1-1.0.1-beta.jar,https://storage-download.googleapis.com/maven-central/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.6.1/iceberg-spark-runtime-3.5_2.12-1.6.1.jar" \
    --properties="{PROPERTIES}"

print("Serverless batch job submitted.")

In [ ]:
# Query the table created by the serverless Spark job
print(f"--- Data returned from Spark query ---")
spark.sql(f"SELECT * FROM {ICEBERG_BIGLAKE_FROM_SPARK}").show()

# Create a new Iceberg table from the results
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {ICEBERG_HIVE_FROM_SPARK}
    USING iceberg
    AS
    SELECT * FROM {ICEBERG_BIGLAKE_FROM_SPARK}
""")

print("--- Iceberg Table on Hive Metastore ---")
spark.sql(f"SELECT * FROM {ICEBERG_HIVE_FROM_SPARK}").show()

---

## [NOT WORKING] Scenario 7: Trino on GCE with BigLake Metastore
#### Notes: Currently there's limited/no header support on the Iceberg connector for Trino 476 *header.x-goog-user-project*, which is needed to connect to *v1beta/restcatalog*.

This scenario demonstrates how to set up a Trino server on a Google Compute Engine (GCE) virtual machine, install Trino using Docker, and configure it to access the BigLake Metastore via the Iceberg REST API.

### Create a GCE VM

First, create a GCE instance with the necessary scopes to interact with BigQuery and Cloud Storage.

In [ ]:
# # Create a GCE instance
# startup_script = """#!/bin/bash
# sudo apt-get update
# sudo apt-get install -y apt-transport-https ca-certificates curl gnupg lsb-release
# curl -fsSL https://download.docker.com/linux/debian/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
# echo \
#   "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/debian \
#   $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
# sudo apt-get update
# sudo apt-get install -y docker-ce docker-ce-cli containerd.io
# sudo systemctl start docker
# sudo systemctl enable docker
# """

# # Write the script to a local file in your notebook environment
# with open("startup.sh", "w") as f:
#     f.write(startup_script)

# print("Startup script saved to startup.sh")

# # Create the GCE instance using the script file
# # This is the corrected command
# !gcloud compute instances create {GCE_INSTANCE_TRINO} \
#     --project={PROJECT_ID} \
#     --zone={GCE_ZONE_TRINO} \
#     --machine-type=e2-medium \
#     --image-family=debian-11 \
#     --image-project=debian-cloud \
#     --scopes=https://www.googleapis.com/auth/cloud-platform \
#     --tags=http-server,https-server \
#     --metadata-from-file=startup-script=startup.sh

# print(f"GCE instance '{GCE_INSTANCE_TRINO}' creation initiated.")

### Install and Configure Trino with Docker

Once the VM is running, SSH into it and install Docker. Then, configure Trino to use the BigLake Metastore.

In [ ]:
# # Step 1: Define the final shell script that manually fetches a token.
# trino_script = f"""#!/bin/bash
# set -e # Exit immediately if a command exits with a non-zero status.

# echo "--- SCRIPT START ---"

# # Install jq to parse JSON
# sudo apt-get update -y && sudo apt-get install -y jq

# # Manually fetch an OAuth2 access token
# echo "Fetching access token..."
# ACCESS_TOKEN=$(curl -s "http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token" -H "Metadata-Flavor: Google" | jq -r .access_token)
# if [ -z "$ACCESS_TOKEN" ]; then
#     echo "FATAL: Failed to fetch access token."
#     exit 1
# fi
# echo "Access token fetched successfully."

# # Create Trino config directories
# mkdir -p ~/trino/etc/catalog

# # Create the Iceberg catalog file with the CORRECT FILENAME and ALL PROPERTIES
# echo "Creating Trino config..."
# cat <<EOF > ~/trino/etc/catalog/{ICEBERG_BIGLAKE_CATALOG}.properties
# connector.name=iceberg
# iceberg.catalog.type=rest
# iceberg.rest-catalog.uri=https://biglake.googleapis.com/iceberg/v1beta/restcatalog
# iceberg.rest-catalog.warehouse=gs://{BUCKET_NAME}/{ICEBERG_BIGLAKE_CATALOG}
# iceberg.rest-catalog.security=GCS_OAUTH
# iceberg.rest-catalog.gcs.oauth.scope=https://www.googleapis.com/auth/cloud-platform
# EOF

# # Create the other Trino config files
# cat <<EOF > ~/trino/etc/jvm.config
# -server
# -Xmx1G
# -XX:+UseG1GC
# -XX:G1HeapRegionSize=32M
# -XX:+UseGCOverheadLimit
# -XX:+ExplicitGCInvokesConcurrent
# -XX:+HeapDumpOnOutOfMemoryError
# -XX:+ExitOnOutOfMemoryError
# EOF
# cat <<EOF > ~/trino/etc/node.properties
# node.environment=production
# node.data-dir=/data/trino
# EOF
# cat <<EOF > ~/trino/etc/config.properties
# coordinator=true
# node-scheduler.include-coordinator=true
# http-server.http.port=8080
# discovery.uri=http://localhost:8080
# EOF

# # Stop and remove any existing Trino container
# sudo docker stop trino || true
# sudo docker rm trino || true

# # Pull and run the Trino container
# echo "Starting Trino container..."
# sudo docker run -d --name trino \
#     -p 8080:8080 \
#     -v ~/trino/etc:/etc/trino \
#     trinodb/trino:476

# echo "--- SCRIPT FINISHED ---"
# echo "Trino should now be starting successfully. Please check 'docker logs trino'."
# """

# # Step 2: Write the script to a local file.
# with open("setup_trino.sh", "w") as f:
#     f.write(trino_script)
# print("Local script 'setup_trino.sh' created.")

# # Step 3: Copy the script to your GCE instance.
# !gcloud compute scp setup_trino.sh {GCE_INSTANCE_TRINO}:~/ --project={PROJECT_ID} --zone={GCE_ZONE_TRINO}
# print(f"Script copied to '{GCE_INSTANCE_TRINO}'.")

# # Step 4: Execute the script on the GCE instance.
# !gcloud compute ssh {GCE_INSTANCE_TRINO} --project={PROJECT_ID} --zone={GCE_ZONE_TRINO} --command="chmod +x ~/setup_trino.sh && bash ~/setup_trino.sh"
# print(f"Trino installation command executed on '{GCE_INSTANCE_TRINO}'.")

### Query Data from Trino

Now, you can connect to the Trino server and query the Iceberg table managed by BigLake Metastore.

In [ ]:
# # Query from Trino
# !gcloud compute ssh {GCE_INSTANCE_TRINO} --project={PROJECT_ID} --zone={GCE_ZONE_TRINO} --command="""
#     docker exec trino trino --catalog {ICEBERG_BIGLAKE_CATALOG} --schema {BIGLAKE_DATASET} --execute "SHOW SCHEMAS;"
#     docker exec trino trino --catalog {ICEBERG_BIGLAKE_CATALOG} --schema {BIGLAKE_DATASET} --execute "SHOW TABLES;"
#     docker exec trino trino --catalog {ICEBERG_BIGLAKE_CATALOG} --schema {BIGLAKE_DATASET} --execute "SELECT * FROM {BIGLAKE_TABLE};"
# """
# print(f"Queries executed on Trino instance '{GCE_INSTANCE_TRINO}'.")

## Scenario 8: Spark on GCE with BigLake Metastore

This scenario demonstrates how to set up a Spark on a Google Compute Engine (GCE) virtual machine, install Spark using Docker, and configure it to access the BigLake Metastore via the Iceberg REST API.

In [ ]:
# Create a GCE instance
startup_script = """#!/bin/bash
sudo apt-get update
sudo apt-get install -y apt-transport-https ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/debian/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/debian \
  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install -y docker-ce docker-ce-cli containerd.io
sudo systemctl start docker
sudo systemctl enable docker
sudo usermod -aG docker $USER
"""

# Write the script to a local file in your notebook environment
with open("startup.sh", "w") as f:
    f.write(startup_script)

print("Startup script saved to startup.sh")

# Create the GCE instance using the script file
# This is the corrected command
!gcloud compute instances create {GCE_INSTANCE_SPARK} \
    --project={PROJECT_ID} \
    --zone={GCE_ZONE_SPARK} \
    --machine-type=e2-medium \
    --image-family=debian-11 \
    --image-project=debian-cloud \
    --scopes=https://www.googleapis.com/auth/cloud-platform \
    --tags=http-server,https-server \
    --metadata-from-file=startup-script=startup.sh

print(f"GCE instance '{GCE_INSTANCE_SPARK}' creation initiated.")

### Install and Configure Spark with Docker

Once the VM is running, SSH into it and install Docker. Then, configure Spark to use the BigLake Metastore.

In [ ]:
# Step 1: Define the shell script for Spark setup.
spark_setup_script = f"""#!/bin/bash
set -e

echo '--- SCRIPT START ---'

# Pull the Spark Docker image
sudo docker pull bitnami/spark:3.5

# Stop and remove any existing Spark container
sudo docker stop spark-biglake || true
sudo docker rm spark-biglake || true

# Run the Spark container in detached mode with proper GCP authentication
sudo docker run -d --name spark-biglake \\
    --user root \\
    --network host \\
    -e SPARK_HOME=/opt/bitnami/spark \\
    -e JAVA_HOME=/opt/bitnami/java \\
    -e HADOOP_USER_NAME=root \\
    -e USER=root \\
    -e HOME=/root \\
    -e GOOGLE_CLOUD_PROJECT={PROJECT_ID} \\
    -e HADOOP_CONF_DIR=/opt/bitnami/spark/conf \\
    -e HADOOP_HOME=/opt/bitnami/spark \\
    bitnami/spark:3.5

# Setup directories, install packages, and test GCP authentication
docker exec spark-biglake bash -c "
    # Create necessary directories
    mkdir -p /tmp/.ivy2
    chmod 777 /tmp/.ivy2

    # Install required Python packages for OAuth2 authentication
    echo 'Installing required Python packages...'
    pip install google-auth google-auth-oauthlib google-auth-httplib2 || echo 'Package installation failed, continuing...'
"

# Create a Python launcher script for interactive Spark session with OAuth2 authentication
cat <<EOF > ~/spark_launcher.py
#!/usr/bin/env python3

import sys
import code
import google.auth
from google.auth.transport.requests import Request
from pyspark.sql import SparkSession

def main():
    print("=== Getting Google Cloud credentials ===")

    # Get default credentials and project
    try:
        credentials, project_id = google.auth.default()
        credentials.refresh(Request())
        access_token = credentials.token
    except Exception as e:
        print(f"Failed to get credentials: {{e}}")
        sys.exit(1)

    print("=== Initializing Spark with BigLake/Iceberg Configuration ===")

    # Configuration
    catalog_name = "{ICEBERG_BIGLAKE_CATALOG}"
    warehouse_bucket = "gs://{BUCKET_NAME}"

    # Create Spark session with OAuth2 token authentication
    spark = SparkSession.builder \\
        .appName("BigLake-Iceberg-Interactive") \\
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \\
        .config("spark.sql.defaultCatalog", catalog_name) \\
        .config(f"spark.sql.catalog.{{catalog_name}}", "org.apache.iceberg.spark.SparkCatalog") \\
        .config(f"spark.sql.catalog.{{catalog_name}}.type", "rest") \\
        .config(f"spark.sql.catalog.{{catalog_name}}.uri", "https://biglake.googleapis.com/iceberg/v1beta/restcatalog") \\
        .config(f"spark.sql.catalog.{{catalog_name}}.warehouse", f"{{warehouse_bucket}}/{{catalog_name}}") \\
        .config(f"spark.sql.catalog.{{catalog_name}}.token", access_token) \\
        .config(f"spark.sql.catalog.{{catalog_name}}.oauth2-server-uri", "https://oauth2.googleapis.com/token") \\
        .config(f"spark.sql.catalog.{{catalog_name}}.header.x-goog-user-project", project_id) \\
        .config(f"spark.sql.catalog.{{catalog_name}}.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO") \\
        .config(f"spark.sql.catalog.{{catalog_name}}.rest-metrics-reporting-enabled", "false") \\
        .getOrCreate()

    print("Spark session created successfully!")
    print(f"Spark version: {{spark.version}}")
    print(f"Application ID: {{spark.sparkContext.applicationId}}")
    print("\\n=== Interactive Spark Session Ready ===")

    # Make spark and sc available in the interactive session
    sc = spark.sparkContext

    # Start interactive Python shell with spark session available
    code.interact(local=locals())

if __name__ == "__main__":
    main()
EOF

# Create a helper script to run the interactive Spark session
cat <<EOF > ~/run_pyspark.sh
#!/bin/bash

echo "Starting interactive Spark session with BigLake/Iceberg configuration..."

# Copy the launcher script into the container
docker cp ~/spark_launcher.py spark-biglake:/tmp/spark_launcher.py

# Run the launcher script using spark-submit with comprehensive GCP authentication
docker exec -it spark-biglake /opt/bitnami/spark/bin/spark-submit \\
    --packages org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1,org.apache.iceberg:iceberg-gcp:1.6.1 \\
    --conf spark.sql.adaptive.enabled=false \\
    --conf spark.jars.ivy=/tmp/.ivy2 \\
    --conf spark.hadoop.fs.gs.auth.service.account.enable=true \\
    --conf spark.hadoop.google.cloud.auth.service.account.enable=true \\
    --conf spark.hadoop.fs.gs.impl=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem \\
    --conf spark.hadoop.fs.AbstractFileSystem.gs.impl=com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS \\
    --conf spark.hadoop.security.authentication=simple \\
    --conf spark.hadoop.security.authorization=false \\
    --conf spark.sql.execution.arrow.pyspark.enabled=false \\
    --driver-java-options "-Dhadoop.home.dir=/opt/bitnami/spark -Duser.name=root -DHADOOP_USER_NAME=root -Dcom.google.cloud.hadoop.util.HttpTransportFactory.type=java_net -Djava.security.auth.login.config=/dev/null -Djava.security.krb5.conf=/dev/null -Dhadoop.security.authentication=simple -Dhadoop.security.authorization=false" \\
    --repositories https://repo1.maven.org/maven2/ \\
    /tmp/spark_launcher.py
EOF

chmod +x ~/run_pyspark.sh

echo '--- SCRIPT FINISHED ---'
echo 'Spark container is running.'
echo 'The interactive session will have spark and sc objects available.'
"""

# Step 2: Write the script to a local file.
with open("setup_spark.sh", "w") as f:
    f.write(spark_setup_script)
print("Local script 'setup_spark.sh' created.")

# Step 3: Copy the script to your GCE instance.
!gcloud compute scp setup_spark.sh {GCE_INSTANCE_SPARK}:~/ --project={PROJECT_ID} --zone={GCE_ZONE_SPARK}
print(f"Script copied to '{GCE_INSTANCE_SPARK}'.")

# Step 4: Execute the script on the GCE instance.
!gcloud compute ssh {GCE_INSTANCE_SPARK} --project={PROJECT_ID} --zone={GCE_ZONE_SPARK} --command=\"chmod +x ~/setup_spark.sh && bash ~/setup_spark.sh\"
print(f"Spark installation command executed on '{GCE_INSTANCE_SPARK}'.")

---

## Step 6: Clean Up Resources

Delete the Dataproc cluster and GCE instance to avoid charges.

In [ ]:
# Delete the Dataproc cluster
!gcloud dataproc clusters delete {CLUSTER_NAME} --region {REGION}

# # Delete the GCE instance
# !gcloud compute instances delete {GCE_INSTANCE_TRINO} --project={PROJECT_ID} --zone={GCE_ZONE_TRINO} --quiet
# print(f"GCE instance '{GCE_INSTANCE_TRINO}' deletion initiated.")
!gcloud compute instances delete {GCE_INSTANCE_SPARK} --project={PROJECT_ID} --zone={GCE_ZONE_SPARK} --quiet
print(f"GCE instance '{GCE_INSTANCE_SPARK}' deletion initiated.")